In [ ]:
import pandas as pd
from transformers import pipeline
import random

# Load the dataset
file_path = 'tapera_label.csv'
dataset = pd.read_csv(file_path)

# Load fill-mask pipeline for Indonesian using IndoBERT
unmasker = pipeline('fill-mask', model='indobenchmark/indobert-base-p1')

In [ ]:
# Filter the dataset for positive sentiment
positive_texts = dataset[dataset['label'] == 'positive']['text'].tolist()

In [3]:
def synonym_replacement(text, unmasker, num_variations=2):
    words = text.split()
    augmented_texts = []

    # Randomize the process by selecting different words to mask in different iterations
    random.shuffle(words)

    for word in words[:num_variations]:
        if word not in text:
            continue
        masked_text = text.replace(word, '[MASK]', 1)
        predictions = unmasker(masked_text)
        for pred in predictions:
            if pred['token_str'] != word:
                new_text = masked_text.replace('[MASK]', pred['token_str'], 1)
                augmented_texts.append(new_text)
                break

        if len(augmented_texts) >= num_variations:
            break

    return augmented_texts

# Apply synonym replacement to positive texts
augmented_texts = []
max_augmentations = 800 
for text in positive_texts:
    results = synonym_replacement(text, unmasker, num_variations=5)  # Adjust num_variations as needed
    augmented_texts.extend(results)
    if len(augmented_texts) >= max_augmentations:
        augmented_texts = augmented_texts[:max_augmentations]
        break

Some weights of BertForMaskedLM were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: All arrays must be of the same length

In [6]:
augmented_df = pd.DataFrame(augmented_texts)
augmented_df.to_csv('augmented_data.csv', index=False)

Augmentation completed with 800 entries.
